# Pull Queries from Postgres as a CSV file

In [1]:
import pandas as pd

#Get the previous date
from datetime import date, timedelta, datetime

#Interacting with Postgres SQL
import psycopg2

import time

#importing database
from database_config import postgres as cfg
from database_config import cust_list

ImportError: cannot import name 'cust_list'

# Establishing Connection to Postgres via psycopg2

In [ ]:
#Setting up the connection to the PostgreSQL -feeddate
conn = psycopg2.connect(database=cfg['database'],user=cfg['user'],password=cfg['password'],host=cfg['host'],port="5432")

#this works with the last days of the month as well. 
todays_date = str(datetime.today())[:10]

In [ ]:
cust_list = pd.read_csv('./data/customer_name_list.csv',index_col='customer_id')

In [5]:
important_customers = cust_list.to_dict(orient='dict')['customer_name']

In [6]:
important_customers['']

{4: 'ldproducts',
 6: 'urbandecay',
 7: 'katesomerville',
 8: 'neimanmarcus',
 9: 'buildcom',
 10: 'swanson',
 11: 'crateandbarrel',
 12: 'landofnod',
 13: 'pier1',
 14: 'toofaced',
 15: 'wasserstrom',
 16: 'cb2',
 18: 'surlatable',
 19: 'sallybeauty',
 20: 'lancome',
 21: 'menswearhouse',
 23: 'staples',
 24: 'hsn',
 25: 'calphalon',
 26: 'discountschoolsupply',
 27: 'columbia',
 29: 'sorel',
 30: 'mountainhardwear',
 32: 'eddiebauer',
 33: 'johnvarvatos',
 34: 'cabelas',
 35: 'kerastase',
 36: 'journeys',
 37: 'restorationhardware',
 38: 'rakuten',
 39: 'josabank',
 40: 'nordstrom',
 45: 'freepeople',
 49: 'francescas',
 50: 'express',
 51: 'stuller',
 52: 'nike',
 53: 'heb',
 55: 'cvs',
 56: 'westelm',
 58: 'zappos',
 59: 'sears',
 60: 'watchdog',
 62: 'homedepot',
 63: 'vitaminshoppe',
 64: 'tractorsupply',
 65: 'zoro',
 67: 'Home Depot - Cleaning',
 68: 'esteelauder',
 69: 'signet',
 71: 'burlington',
 72: 'officedepot',
 74: 'roadrunner',
 75: 'dickssportinggoods',
 77: 'containe

## Query

In [7]:
company_na_reports = (  
'''SELECT 
    ct."id" AS curation_task_id, 
    ct.customer_id, 
    ct.resolution, 
    --- truncated the timestamp from the date for aggregation.
    date_trunc('day',ct.started_at) as "started_at",
    cpf.product_id,
    cpf."name" AS attribute_name, 
    cpf."Total_NAs", 
    cpf.curation_tasks_count
FROM "curation_tasks" AS ct

INNER JOIN 
    (SELECT 
        cpf.curation_task_id,
        cpf.customer_id,
        cpf.product_id,
        cpf.name, 
        SUM(CASE WHEN cpf.value = 'n/a' THEN 1 ELSE 0 END) as "Total_NAs",
        COUNT(cpf.id) AS curation_tasks_count
    FROM "public"."curated_product_fields"  as cpf
    WHERE customer_id = {}
    GROUP BY
        cpf.curation_task_id,    
        cpf.customer_id,
        cpf.product_id,
        cpf.name
    ) AS cpf
ON ct.id = cpf.curation_task_id

--- specific resolution that are not taken care of my rules or bulk
WHERE (ct.resolution IS NULL OR ct.resolution = 'misclassified') 
AND ct.customer_id = {}
ORDER BY started_at;
''')

In [8]:
def single_customer(customer_id):
    data = pd.read_sql(company_na_reports.format(customer_id,customer_id),conn)
    if data.shape[0] != 0:
        data['started_at'] = data['started_at'].dt.date
        data['customer_name']= important_customers[customer_id]
    return data

In [9]:
data = single_customer(21)

In [10]:
data.to_csv('menswearhouse.csv')

In [38]:
master_data = pd.DataFrame()
for cust in important_customers:
    #Returning the data in pandas to export it as a CSV.
    data = pd.read_sql(company_na_reports.format(cust,cust),conn)
    if data.shape[0] != 0:
        data['started_at'] = data['started_at'].dt.date
        data['customer_name']= important_customers[cust]
    else:
        print('No data available in your time frame for Cust_id ',cust)
        pass
    master_data = master_data.append(data,sort=False)
    print(cust, 'query and data clean completed')
    time.sleep(15)

No data available in your time frame for Cust_id  4
4 query and data clean completed
No data available in your time frame for Cust_id  6
6 query and data clean completed
No data available in your time frame for Cust_id  7
7 query and data clean completed
No data available in your time frame for Cust_id  8
8 query and data clean completed
9 query and data clean completed
No data available in your time frame for Cust_id  10
10 query and data clean completed
11 query and data clean completed
No data available in your time frame for Cust_id  12
12 query and data clean completed
13 query and data clean completed
No data available in your time frame for Cust_id  14
14 query and data clean completed
No data available in your time frame for Cust_id  15
15 query and data clean completed
16 query and data clean completed
No data available in your time frame for Cust_id  18
18 query and data clean completed
No data available in your time frame for Cust_id  19
19 query and data clean completed
20 

In [39]:
master_data.shape

(337666, 9)

In [40]:
master_data.head()

,curation_task_id,customer_id,resolution,started_at,product_id,attribute_name,Total_NAs,curation_tasks_count,customer_name
0,95157248,9,None,2018-10-18,247541549,range_hood_accessory_type,1,1,buildcom
0,98736130,11,None,2018-10-18,285933017,shape,0,1,crateandbarrel
1,98736205,11,None,2018-10-18,285933060,shape,0,1,crateandbarrel
2,96873410,11,None,2018-10-18,285173942,product_type,0,1,crateandbarrel
3,98736197,11,None,2018-10-19,285933060,width,0,1,crateandbarrel


In [41]:
if master_data.shape[0] != 0:
    master_data.to_csv('./data/Weekly_NA_report.csv',index=False)

# Querying n/a Product data

In [42]:
query_product = (
'''SELECT
    p.customer_id,
    pb.bucket_id,
    b.name as bucket_name,
    p.id as product_id, 
    p.name as product_name,
    active, 
    p.external_id,
    p.image_url
FROM "products" AS p  

INNER JOIN products_buckets as pb
ON p.id = pb.product_id
INNER JOIN buckets as b
ON pb.bucket_id = b.id
Where active = 't'
AND p.customer_id  = {}
ORDER BY p.id
'''
)

In [44]:
master_data_prod = pd.DataFrame()
for cust in important_customers:
    #Returning the data in pandas to export it as a CSV.
    data_prod = pd.read_sql(query_product.format(cust),conn)
    if data_prod.shape[0] == 0:
        print('No data_prod available in your time frame for Cust_ID ',cust)
    else:
        pass
    master_data_prod = master_data_prod.append(data_prod,sort=False)
    print('Cust_id ',cust, 'Product query and data clean completed')
    time.sleep(10)

No data_prod available in your time frame for Cust_ID  4
Cust_id  4 Product query and data clean completed
Cust_id  6 Product query and data clean completed
No data_prod available in your time frame for Cust_ID  7
Cust_id  7 Product query and data clean completed
No data_prod available in your time frame for Cust_ID  8
Cust_id  8 Product query and data clean completed
Cust_id  9 Product query and data clean completed
No data_prod available in your time frame for Cust_ID  10
Cust_id  10 Product query and data clean completed
Cust_id  11 Product query and data clean completed
Cust_id  12 Product query and data clean completed
Cust_id  13 Product query and data clean completed
No data_prod available in your time frame for Cust_ID  14
Cust_id  14 Product query and data clean completed
No data_prod available in your time frame for Cust_ID  15
Cust_id  15 Product query and data clean completed
Cust_id  16 Product query and data clean completed
Cust_id  18 Product query and data clean complet

In [45]:
if master_data_prod.shape[0] != 0:
    master_data_prod.to_csv('./data/Weekly_products.csv',index=False)

In [46]:
master_data.shape[0],master_data_prod.shape[0]

(337666, 1014679)